In [ ]:
!pip install langchain-community         

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.document_loaders.csv_loader import CSVLoader   
from langchain_community.embeddings import HuggingFaceEmbeddings 
from langchain.vectorstores import FAISS   
from charset_normalizer import from_path
import pandas as pd 
from langchain_community.embeddings import HuggingFaceEmbeddings  
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline 
from langchain_community.llms import HuggingFacePipeline  

2025-08-22 18:23:29.073250: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755887009.267013      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755887009.327254      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
csv_path = "/kaggle/input/q-ands-a/codebasics_faqs.csv" 
  
# 1) Detect encoding
enc = (from_path(csv_path).best() or from_path(csv_path).first()).encoding or "latin1" 

# 2) Read CSV (handle both new/old pandas) 
read_kwargs = {"encoding": enc}
try:
    df = pd.read_csv(csv_path, on_bad_lines="skip", **read_kwargs)
except TypeError:  
    # for older pandas 
    df = pd.read_csv(csv_path, error_bad_lines=False, **read_kwargs) 
   
# 3) Turn into Documents
try:  
    from langchain_community.document_loaders import DataFrameLoader 
except ImportError:  
    # Fallback if langchain_community isn't available
    from langchain.document_loaders import DataFrameLoader

loader = DataFrameLoader(df, page_content_column="prompt")  # <- use your text column
docs = loader.load() 

docs[:2]  # quick peek  

In [3]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={"device": "cpu"})
vectordb = FAISS.from_documents(documents=docs, embedding=embeddings)      

/tmp/ipykernel_36/368142363.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={"device": "cpu"})


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

NameError: name 'docs' is not defined